In [1]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.pinecone import Pinecone
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint as HuggingFaceHub
from dotenv import load_dotenv
from pinecone import Pinecone as pc
from pinecone import PodSpec
import os

from data_chunking import datachunk

load_dotenv()

from langchain.retrievers.multi_query import MultiQueryRetriever

c:\Users\szh\Desktop\EER\EER-chatbot-UI\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
embeddings = HuggingFaceEmbeddings()
    
pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=embeddings)

index_name = "eerbot"
environment = "gcp-starter"
spec = PodSpec(environment=environment)

docsearch = Pinecone.from_existing_index(index_name, embeddings)
print("Using existing Pinecone index")


Using existing Pinecone index


In [3]:
llm = HuggingFaceHub(
            repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
            temperature=1.0,
            top_p=0.8,
            top_k=50,
            huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
        )

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [4]:
multiquery_retriever_llm = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(), llm=llm
)

In [6]:
docsearch.as_retriever().invoke("What is EER?")

[Document(page_content="Helene Nymann, 2022-10-13, 19:25, that's where EER comes into the picture. No more I was thinking of like sharing perspectives or something, you know, like, what would that would that do? Or you know, that have visitors pose a question before they come in or think about, you know, could something be done that little", metadata={'source': 'data/reformatted_transcripts/2022-10-13 09.05.42_rf.txt'}),
 Document(page_content="World Wide WebBackINviteME Online'EER and the More-than-human'workshopEER | INviteME Onlinehttps://www.eer.info/activities/inviteme-online\n3 of 1028/01/2024, 18:10", metadata={'page': 2.0, 'source': 'data/EER-site-pages-pdf/EER INviteME Online.pdf'}),
 Document(page_content="Sophie Erlund, 2022-01-20, 38:52, Okay, I just wanted to say that, I think that EER, as a project, of course, it's built into it is also this thing about iterations, right? Looking at and reflecting, right, experiencing experimenting with something people haven't experience

In [8]:
multiquery_retriever_llm.invoke("Summarize how the discussions in meeting transcripts in 2021 likely influenced the conversations held in 2022.")

[Document(page_content="Unknown Speaker, 2022-08-25, 44:31, It's just not much to to discuss here. It's just a matter of continuing? Yeah.\nUnknown Speaker, 2022-08-25, 44:44, Did you find ways of because we were setting aside time for interviews or something like that? Yep. Is that still in place?", metadata={'source': 'data/reformatted_transcripts/2022-08-25 10.29.41_rf.txt'}),
 Document(page_content="Karsten Olsen, 2023-12-07, 13:15, But um, I, you know, you're absolutely right. I was also just thinking that the questions you just mentioned, that you would like to, it's good for summarizing research, asking questions about our research. That particular question is the same as when we started discussing the workshop quite some time ago. So fundamentally, that question has changed, but I'm not saying it could probably change and new,", metadata={'source': 'data/reformatted_transcripts/2023-12-07 09.04.36_rf.txt'}),
 Document(page_content="Andreas Roepstorff, 2021-11-25, 55:15, We can'

In [ ]:
default_template = """
   You are a chatbot working for the Experimenting Experiencing Reflecting (EER) Project, a research endeavor investigating the connections between art and science.
    You have access to a collection of documents, including descriptions of research activities, meeting transcripts, and other relevant materials.
    Your main task is to help the user explore and reflect on the EER project.
    All questions should pertain to the EER Project unless specified otherwise.
    When possible, please cite source documents at the end of your answer.
    """
    
template_end = """
    Context: {context}
    Question: {question}
    Answer: 
    
    """

In [ ]:
rag_chain_1 = (
    {"context": multiquery_retriever_llm, "question": RunnablePassthrough()}
    | PromptTemplate(template = default_template + template_end, input_variables=["context", "question"])
    | llm
    | StrOutputParser()
)

In [ ]:
retriever_method = docsearch.as_retriever()

rag_chain_2 = (
    {"context": retriever_method, "question": RunnablePassthrough()}
    | PromptTemplate(template = default_template + template_end, input_variables=["context", "question"])
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain_1.invoke("what is EER")

In [ ]:
rag_chain_2.invoke("what is EER")

In [ ]:
class ChatBot():
    
    embeddings = HuggingFaceEmbeddings()
    
    pinecone_instance = pc(api_key=os.getenv('PINECONE_API_KEY'), embeddings=embeddings)
    
    index_name = "eerbot"
    environment = "gcp-starter"
    spec = PodSpec(environment=environment)
    
    if index_name not in pinecone_instance.list_indexes().names():
        docs = datachunk()
        pinecone_instance.create_index(name=index_name, metric="cosine", dimension=768, spec=spec)
        docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)
        print("Created new Pinecone index and loaded documents")
    else:
        docsearch = Pinecone.from_existing_index(index_name, embeddings)
        print("Using existing Pinecone index")
    
    llm = None  # Define llm, it will be initialized in __init__

    print("HuggingFace Hub initialized...")

    default_template = """
    You are a chatbot working for the Experimenting Experiencing Reflecting (EER) Project, a research endeavor investigating the connections between art and science.
    You have access to a collection of documents, including descriptions of research activities, meeting transcripts, and other relevant materials.
    Your main task is to help the user explore and reflect on the EER project.
    All questions should pertain to the EER Project unless specified otherwise.
    When possible, please cite source documents at the end of your answer.
    """
    
    template_end = """
    Context: {context}
    Question: {question}
    Answer: 
    
    """
    
    def __init__(self, custom_template=None, repo_id=None, temperature=0.8, retriever_method='docsearch'):
        if custom_template:
            self.template = custom_template
        else:
            self.template = self.default_template
        
        self.temperature = temperature
        self.repo_id = repo_id
        self.retriever_method = retriever_method
        
        # Initialize llm with repo_id
        self.llm = HuggingFaceHub(
            repo_id=self.repo_id,
            temperature=temperature,
            top_p=0.8,
            top_k=50,
            huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
        )

        if retriever_method == "docsearch":
            retriever_method = docsearch.as_retriever()
        elif retriever_method == "multiquery_retriever_llm":
            multiquery_retriever_llm = MultiQueryRetriever.from_llm(retriever=docsearch.as_retriever(), llm=self.llm)
            retriever_method = multiquery_retriever_llm


        # Initialize the chain
        self.rag_chain = (
            {"context": retriever_method, "question": RunnablePassthrough()}
            | PromptTemplate(template=self.template+self.template_end, input_variables=["context", "question"])
            | self.llm
            | StrOutputParser()
        )

        print("Chain assembled...")

In [1]:
from main import ChatBot

c:\Users\szh\Desktop\EER\EER-chatbot-UI\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Using existing Pinecone index


In [2]:
bot = ChatBot(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.8, retriever_method='multiquery_retriever_llm')

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful
Chain assembled...


In [3]:
bot.rag_chain.invoke("what is EER")

"\n    EER stands for Experimenting Experiencing Reflecting, which is a research project investigating the connections between art and science. The project involves iterations of experiences, reflections, and experiments with various art forms and concepts. The EER project also includes workshops, such as the 'EER and the More-than-human' workshop held in August 2021 at Studio Olafur Eliasson in Copenhagen. Additionally, the EER project has an online presence, with resources and activities available on the EER website (<https://www.eer.info/>).\n\n    References:\n    - data/reformatted_transcripts/2022-01-20 09.03.52_rf.txt\n    - data/EER-site-pages-pdf/EER INviteME Online.pdf\n    - data/EER-site-pages-pdf/EER Performative scores.pdf"

In [ ]:
bot.rag_chain.invoke("what is EER")